In [1]:
import numpy as np

In [2]:
def set_zero_flag(inp):
    if((inp==0).any()):
        zero_flag = 1
    else:
        zero_flag = 0
    return zero_flag

In [3]:
def water_stored_per_unit(narray1,narray2):
    count =np.zeros(narray1.shape,dtype = 'i4')
    cnt = np.zeros(4,dtype='i4')
    for i in range(narray1.shape[0]):
        for j in range(narray1.shape[1]):
            #print(narray2[0:i,j+1])
            cnt = np.zeros(4)
            if(narray1[i,j]>0):
                cnt[0] += ((narray2[:i+1,j+1]-narray1[i,j]).max())
                cnt[2] += ((narray2[i+1,:j+1]-narray1[i,j]).max())
                cnt[1] += (narray2[i+1,j+2:]-narray1[i,j]).max()
                cnt[3] += (narray2[i+2:,j+1]-narray1[i,j]).max()
            if np.size(cnt[cnt>0])>0:
                count[i,j] = cnt[cnt>0].min()
    return count

In [4]:
def reduce_platform(narray):
    n = np.shape(narray)
    narray2 = np.zeros((n[0]-2,n[1]-2),dtype = 'i4')
    for i in range(n[0]-2):
        for j in range(n[1]-2):
            narray2[i,j]=narray[i+1,j+1]
    return narray2

In [5]:
def modify_platform(inp):
    s = inp.shape
    if(s[0]<s[1]):
        temp = np.repeat(inp[s[0]-1,:],s[1]-s[0]).reshape((s[1]-s[0],s[1]),order='F') 
        mod_inp = np.concatenate((inp,temp),axis = 0)
    elif(s[0]>s[1]):
        temp = np.repeat(inp[:,s[1]-1],s[0]-s[1]).reshape((s[0],s[0]-s[1]),order='F')
        mod_inp = np.concatenate((inp,temp),axis = 1)
    else:
        mod_inp = inp
    return mod_inp

In [17]:
def map_drain_towers(mod_inp):
    #checks whether the tower is connected to drain or not
    drain_platform = np.ones(mod_inp.shape,dtype='i4')#Platform to track the drain points
    if(set_zero_flag(mod_inp)==1):
        ind = np.argwhere(mod_inp==0)
        for l in range(ind.shape[0]):
            index = ind[l]
            i,j = index[0],index[1]
            drain_platform[i,j] = 0
        for i in range(1,inp.shape[0]-1):
            for j in range(1,inp.shape[1]-1):
                if ((drain_platform[i,j]!=0) and (mod_inp[i,j]>=mod_inp[i,j-1]) and (drain_platform[i,j-1]==0) ):
                    drain_platform[i,j]=0
                if ((drain_platform[i,j]!=0) and (mod_inp[i,j]>=mod_inp[i,j+1]) and (drain_platform[i,j+1]==0) ):
                    drain_platform[i,j]=0
                if ((drain_platform[i,j]!=0) and (mod_inp[i,j]>=mod_inp[i-1,j]) and (drain_platform[i-1,j]==0) ):
                    drain_platform[i,j]=0
                if ((drain_platform[i,j]!=0) and (mod_inp[i,j]>=mod_inp[i+1,j]) and (drain_platform[i+1,j]==0) ):
                    drain_platform[i,j]=0
    return drain_platform

In [23]:
def WaterStoredInPlatform(platform):
    s = platform.shape
    mod_platform = modify_platform(platform)
    drain_platform = map_drain_towers(mod_platform)
    reduced_drain_platform = reduce_platform(drain_platform)
    reduced_platform = reduce_platform(mod_platform)
    individ_counts = water_stored_per_unit(reduced_platform,mod_platform)
    mod_individ_counts = (reduced_drain_platform*individ_counts)
    if(s[0]<s[1]):
        mod_actual_counts = mod_individ_counts[:s[0]-s[1],:]
    elif(s[0]>s[1]):
        mod_actual_counts = mod_individ_counts[:,:s[1]-s[0]]
    else:
        mod_actual_counts = mod_individ_counts    
    return mod_actual_counts.sum()

In [26]:
inp = np.array([[5,5,5,5,5],[9,1,1,1,5],[5,1,5,1,5],[5,1,1,1,5],[5,5,5,5,5]],dtype ='i4')
water_stored = WaterStoredInPlatform(inp)
water_stored

32